In [107]:
import cv2

In [108]:
import numpy as np
face_cascade=cv2.CascadeClassifier("/content/drive/My Drive/haarcascade_frontalface_alt.xml")

# Data of Images 

In [109]:
train_path="/content/drive/My Drive/5-celebrity-faces-dataset/data/train"
test_path="/content/drive/My Drive/5-celebrity-faces-dataset/data/val"

In [110]:
pip install MTCNN

In [111]:
from os import listdir
from os.path import isdir
from PIL import Image
import matplotlib.pyplot as plt
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN
import cv2
import time
from google.colab.patches import cv2_imshow

# Extract Images from Folder with location of each images and split in training and testing

In [112]:
 
def extract_face(image):
    img = cv2.imread(image, cv2.IMREAD_UNCHANGED)
    im=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
 
    pixel=asarray(im)
    detector=MTCNN()
    results=detector.detect_faces(pixel)
    
    if len(results)>0:
      
      x1,y1,weight,height=results[0]["box"]
      x1,y1=abs(x1),abs(y1)
      x2,y2=x1+weight,y1+height
      face=pixel[y1:y2,x1:x2]
    
    # image = Image.fromarray(face)
    
      image = cv2.resize(face,(160,160))
      face_array = asarray(image)
    
      return face_array
   
      
 
 
 
def load_faces(path):
    faces=list()
    for img in listdir(path):
        image=path+img
        
        face=extract_face(image)
        faces.append(face)
    return faces
 
 
def load_dataset(train_path):
    x,y=list(),list()
    for subfolder in listdir(train_path):
        path=train_path+subfolder
 
        if not isdir(path):
            continue
        
        faces=load_faces(path+"/")
        labels=[subfolder for i in range(len(faces))]
        print('>loaded %d examples for class: %s' % (len(faces), subfolder))
        x.extend(faces)
        y.extend(labels)
    return asarray(x),asarray(y)

In [113]:
train_x,train_y=load_dataset(train_path+"/")

>loaded 17 examples for class: madonna
>loaded 16 examples for class: Liton
>loaded 17 examples for class: mindy_kaling
>loaded 21 examples for class: jerry_seinfeld
>loaded 16 examples for class: elton_john
>loaded 14 examples for class: ben_afflek


In [114]:
test_x,test_y=load_dataset(test_path+"/")

>loaded 5 examples for class: Liton
>loaded 5 examples for class: mindy_kaling
>loaded 5 examples for class: madonna
>loaded 5 examples for class: jerry_seinfeld
>loaded 5 examples for class: elton_john
>loaded 5 examples for class: ben_afflek


In [115]:
print(train_x.shape)
print(train_y.shape)

(101, 160, 160, 3)
(101,)


In [116]:
print(test_x.shape)
print(test_y.shape)

(30, 160, 160, 3)
(30,)


# Saved into zip format images and load this data for train test split

In [117]:
savez_compressed("faceRecognitionload_faces.npz",train_x,train_y,test_x,test_y)

In [118]:
from numpy import load,savez_compressed,asarray,expand_dims

In [119]:
from keras.models import load_model
data=load("faceRecognitionload_faces.npz")

In [120]:
#arr_0 to arr_3 it is a format of zip comprezzed file
train_x,train_y,test_x,test_y=data["arr_0"],data["arr_1"],data["arr_2"],data["arr_3"]

In [121]:
#let this is our test data 
#5 is a selection
# s=test_x[5]
# import matplotlib.pyplot as plt
 
# plt.imshow(s)

# Predefined Model and load model

In [122]:
model=load_model("/content/drive/My Drive/facenet_keras.h5")

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [123]:
print("train_x shape",train_x.shape)
 
print("train_y shape",train_y.shape)
print("test x shape",test_x.shape)
print("test y shape",test_y.shape)

train_x shape (101, 160, 160, 3)
train_y shape (101,)
test x shape (30, 160, 160, 3)
test y shape (30,)


In [124]:
test_x[0].shape

(160, 160, 3)

# Pixel of image array ,it convert it into embedding vector or standarization of data of images

In [125]:
import numpy as np
def get_embedding(model,pixel):
    face_pixel=pixel.astype("float32")
    mean,std=face_pixel.mean(),face_pixel.std()
    # print("Face Pixel",face_pixel)
    #below we are standarising our fixel 
    face_pixel=(face_pixel-mean)/std
    # print("Facce PiXEL 2",face_pixel.shape)
    sample=expand_dims(face_pixel,axis=0)
 
    # print("Sample",sample.shape)
    prediction=model.predict(sample)
    # print(prediction.shape)
    #(1,128)
    return prediction[0]
 
 
#transform your image into vector first train 
newtrainx=list()
for face_pixel in train_x:
    
    #we have resize it because facenet keras has shape 160,160 that's i resize it
#     face_pixel=np.resize(face_pixel,(160,160,3))
#     face_pixel=asarray(face_pixel)
    
    embedding=get_embedding(model,face_pixel)
    newtrainx.append(embedding)
newtrainx=asarray(newtrainx)
#then testing data
newtestx=list()
for face_pixel2 in test_x:
#     face_pixel2=np.resize(face_pixel2,(160,160,3))
#     face_pixel2=asarray(face_pixel2)
    embedding2=get_embedding(model,face_pixel2)
    newtestx.append(embedding2)
newtestx=asarray(newtestx)

In [126]:
savez_compressed("faceRecognitionload_faces2.npz",newtrainx,train_y,newtestx,test_y)

In [127]:
newtestx.shape

(30, 128)

In [128]:
newtestx[0].shape

(128,)

# Capture Image through webcam ,we can't open open through opencv ,this  api created by google

In [129]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
 
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);
 
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
 
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();
 
      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
 
      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);
 
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [130]:
# filename=take_photo()

In [131]:
# filename
# photo.jpg

# prediction

In [150]:
# develop a classifier for the 5 Celebrity Faces Dataset
from random import choice
from numpy import load
from numpy import expand_dims
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot
# load faces
# data = load('faceRecognitionload_faces.npz')
#below is a testing data -test_x
# testX_faces = data['arr_2']
# load face embeddings
data = load('faceRecognitionload_faces2.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)
# test model on a random example from the test dataset
 
# selection = choice([i for i in range(testX.shape[0])])
# print("selection",selection)
#5 ,it is for random image selection
#below is a test image for plotting
# choose="Image"
 
#Just for displaying test data
# image="/content/drive/My Drive/IMG_20190328_142112_474.jpg"
 
from IPython.display import Image
try:
  filename = take_photo()
  #below is a file name which is stored in jpg extension 
  # print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
 
  # imgr=cv2.imread(filename, cv2.IMREAD_UNCHANGED)
  # cv2_imshow(imgr)
  
  # display(Image(filename))
  
except Exception as err:
  
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))
#image name is given
image="photo.jpg"
a=extract_face(image)
random_face_pixels=a.any()
# print(random_face_pixels)
model1=load_model("/content/drive/My Drive/facenet_keras.h5")

#if image is not detected it means is None.If detected it has some value
if random_face_pixels!=None:
  s=get_embedding(model1,a)
  random_face_emb = s
  #this is aclass if selecction is 5 it will be 5th
  # import random
  # j=random.randint(0,30)
  random_face_class = testy[0]
 
  random_face_name = out_encoder.inverse_transform([random_face_class])
  # print(random_face_name,"name of the person")
  # prediction for the face
  samples = expand_dims(random_face_emb, axis=0)
  #samples (1, 128)
  # print("samples",samples.shape)
  yhat_class = model.predict(samples)
  print("class",yhat_class)
  #class (1,) class [0]
  yhat_prob = model.predict_proba(samples)
  #[[9.99999744e-01 3.81907437e-08 9.86691609e-08 2.36270613e-08
  # 3.27224649e-08 6.26039307e-08]]
  # print(yhat_prob)
  #prob (1, 6)
  # get name
  class_index = yhat_class[0]
  #0 is a index
  # print(class_index)
  class_probability = yhat_prob[0,class_index] * 100
  predict_names = out_encoder.inverse_transform(yhat_class)
  #print(predict_names) ['Liton']
  #It will tell you the name of the predict class.
  print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
  # print('Expected: %s' % random_face_name[0])
  if predict_names[0]==random_face_name[0]:
    print("Access Granted : Face Recognized")
 
  else:
    print("Access Denied : Not Recognized ")
else:
  print("Face is Not detected ")
# plot for fun
# pyplot.imshow(random_face_pixels)
# title = '%s (%.3f)' % (predict_names[0], class_probability)
# pyplot.title(title)
# pyplot.show()

<IPython.core.display.Javascript object>

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


class [4]
Predicted: madonna (100.000)
Access Denied : Not Recognized 
